In [8]:
import pathlib as pl
import re as re
import collections as col

local_run = True

local_imbb_mount = pl.Path("/mounts/hilbert/project/projects/medbioinf")
remote_imbb_path = pl.Path("/gpfs/project/projects/medbioinf")

data_root_path = local_imbb_mount if local_run else remote_imbb_path

sample_fofn_path = data_root_path.joinpath("data/00_RESTRUCTURE/sample-centric")

samples = []
for sample_folder in sample_fofn_path.iterdir():
    if sample_folder.is_symlink():
        continue
    sample_name = sample_folder.name
    if sample_name.startswith("GM") or sample_name == "HG002":
        continue
    hifi_data = list(sample_folder.glob("*hifi*fastq*.fofn"))
    ont_data = list(sample_folder.glob("*ont*fastq*.fofn"))
    if hifi_data and ont_data:
        samples.append(sample_name)
        
print("\n".join(sorted(samples)))
    
    

HG00268
HG00512
HG00731
HG00733
HG01457
HG02106
HG02282
HG02666
HG03248
HG03520
HG03807
HG04217
NA18989
NA19239
NA19317
NA19331
NA19347
NA19384
NA19434
NA19836
NA24385
